In [11]:
from dataclasses import dataclass
from langchain.tools import tool
from langchain.messages import AnyMessage
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import dynamic_prompt, ModelRequest, before_model, after_model
from langgraph.runtime import Runtime
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

@dataclass
class Context:
    user_name: str

# Dynamic prompts
@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context.user_name  
    system_prompt = f"You are a helpful assistant. Address the user as {user_name}."
    return system_prompt

# Before model hook
@before_model
def log_before_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Processing request for user: {runtime.context.user_name}")  
    return None

# After model hook
@after_model
def log_after_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Completed request for user: {runtime.context.user_name}")  
    return None


@tool
def add(a: float, b: float)->float:
    """
    Performs addition of two number
    """
    return a+b


agent_prompt = """You are an addition operation performing agent, Evaluate the user's expression
Ensure you always respond user with his name"""

llm = init_chat_model(
    model = "gpt-4o-mini",
    model_provider='openai',
)

agent = create_agent(
    model=llm,
    tools=[add],
    middleware=[dynamic_system_prompt, log_before_model, log_after_model],  
    context_schema=Context,
    system_prompt = agent_prompt,
)

agent.invoke(
    {"messages": [{"role": "user", "content": "1112+2223+7333?"}]},
    context=Context(user_name="Narasimhulu")
)

Processing request for user: Narasimhulu
Completed request for user: Narasimhulu
Processing request for user: Narasimhulu
Completed request for user: Narasimhulu
Processing request for user: Narasimhulu
Completed request for user: Narasimhulu


{'messages': [HumanMessage(content='1112+2223+7333?', additional_kwargs={}, response_metadata={}, id='fde23c66-0493-4cdc-b47b-056167ffb0c1'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 70, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b547601dbd', 'id': 'chatcmpl-CgSJVrXwyod6hGJ1YkFSUGX6EZZ7b', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--471e2702-560e-4335-b55b-1ced9f8d3a17-0', tool_calls=[{'name': 'add', 'args': {'a': 1112, 'b': 2223}, 'id': 'call_ZjFUqeI1zWhx3UQkASiNEloJ', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 7333, 'b': 0}, 'id': 'call_f1s5Z87T3GZRAJJQfYp7xpLB',